## YoLov3 Object detection on image

#### Importing Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#### Reading image file with its shape

In [2]:
img = cv2.imread("office.jpg")
cv2.imshow("office", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
print(img.shape)

height, width, _ = img.shape

(415, 737, 3)


#### Reading network with pre-trained weights and configurations of YoLov3 tiny

In [4]:
net = cv2.dnn.readNet("yolov3-tiny.weights", "yolov3.cfg")

#### Reading classes set from coco file

In [5]:
classes = []

with open('coco.names', 'r') as f:
    classes = f.read().splitlines()

#print(classes)

In [6]:
len(classes)

80

#### Preprocessing of Image

In [7]:
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB=True, crop=False)

In [8]:
net.setInput(blob)

output_layer_names = net.getUnconnectedOutLayersNames()

layerOutputs = net.forward(output_layer_names)

#### reading output of layes with RGB images

In [9]:
"""
for b in blob:
    for n, img in enumerate(b):
        cv2.imshow(str(n), img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
"""

'\nfor b in blob:\n    for n, img in enumerate(b):\n        cv2.imshow(str(n), img)\n        cv2.waitKey(0)\n        cv2.destroyAllWindows()\n'

In [10]:
#print(layerOutputs)

#### make bounding boxes around detected objects and and represent with label and confidence

In [11]:
boxes = []
confidences = []
class_ids = []

for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
print(boxes)
print(" ")
print(confidences)
print(" ")
print(len(boxes))
print(" ")

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
print(indexes.flatten())

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(boxes),3))

#if len(indexes)>0:
for i in indexes.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    confidence = str(round(confidences[i], 2))
    color = colors[i]
    cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
    cv2.putText(img, label+" "+ confidence, (x, y+20), font, 1, (255,0,255), 1)


[[219, 14, 190, 142], [310, 167, 148, 42], [466, 173, 17, 8], [360, 181, 56, 30]]
 
[0.690402626991272, 0.6304581165313721, 0.7946690917015076, 0.9340662956237793]
 
4
 
[3 2 0 1]


#### Visualize detected objects

In [12]:
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()